In [177]:
import numpy as np
import random
import math
from operator import mul,neg

from EstrategiaParticionado import ValidacionCruzada
from datos import Datos
from Clasificador import Clasificador

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split


vector = np.array([1,17.99,10.38,122.8,1001,0.1184,0.2776,0.3001,0.1471,0.2419,0.07871,1.095,0.9053,8.589,153.4,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019,0.1622,0.6656,0.7119,0.2654,0.4601,0.1189])


In [2]:
def producto_escalar(w,x):
    return sum(map(mul,w,x))


In [263]:
def calcular_sigmoidal (a):
    return pow(1 + pow(math.e,neg(math.sqrt(a))),-1)


In [4]:
def generar_vector(d):
    return np.array([ random.uniform(-1,1) for i in range(d) ])


In [262]:
pe = producto_escalar(generar_vector(len(vector)),vector)
print(pe)
print(calcular_sigmoidal(pe))

175.773299887


ValueError: math domain error

In [173]:
class ClasificadorRegresionLogistica(Clasificador):
    w = np.array([])
    n = 0.0 #cte aprendizaje
    num_epocas = 0

    def __init__(self,num_epocas, n):
        super(ClasificadorRegresionLogistica, self).__init__()
        self.num_epocas = num_epocas
        self.n = n
    def __generar_vector_nuevo(self,dato,sigmoidal,clase):
        factor = self.n * (sigmoidal - clase)
        return self.w - np.array(list(map(lambda x : x * factor, dato)))
    def entrenamiento(self, datosTrain, atributosDiscretos, diccionario):
        self.w = generar_vector(len(atributosDiscretos))
        for epoca in range(self.num_epocas):
            for dato in datosTrain:
                pe = producto_escalar(self.w,dato[:-1])
                print(pe)
                sigmoidal = calcular_sigmoidal(pe)
                self.w = self.__generar_vector_nuevo(dato,
                                                     sigmoidal,
                                                     dato[-1])
        return self.w
    def __clasifica_dato(self,dato_test):
        return int(calcular_sigmoidal(producto_escalar(self.w,dato_test)) > 0.5)
    def clasifica (self, datosTest, atributosDiscretos, diccionario):
        return list(map(self.__clasifica_dato, datosTest))


In [14]:
wdbc = Datos('../ficheros/ConjuntosDatos/wdbc.data',True)

In [114]:
estrategia = ValidacionCruzada(10)
sklearn_cv = KFold(n_splits=10,shuffle=True)
clasificador_sklearn = LogisticRegression() 

In [115]:
#clasificador = ClasificadorRegresionLogistica(10,1)

scores = cross_val_score(clasificador_sklearn, wdbc.datos[:,:-1],wdbc.datos[:,-1],cv = sklearn_cv)
print(np.mean(scores))

0.565758145363


In [136]:
X_train,X_test,y_train,y_test = train_test_split(wdbc.datos[:,:-1],
                                                 wdbc.datos[:,-1], 
                                                 test_size = 0.2)
clasificador_sklearn.fit(X_train,y_train)
print(clasificador_sklearn.score(X_test,y_test))

0.508771929825


In [174]:
clasificador = ClasificadorRegresionLogistica(5,0.1)

In [175]:
errores = clasificador.validacion(estrategia,wdbc,clasificador)

print(1 - np.mean(errores))

-330.684133607
172159.21589
182683.473886
154239.202912
185089.303568
148547.471723
151235.138341
173899.489943
176514.632831
137383.100589
200479.372005
165688.75762
177804.009308
134379.506146
139903.023355
170220.712312
191285.603492
177912.919077
219398.237241
206271.387935
-17697.2809686


OverflowError: math range error